In [24]:
# 檢查 CUDA 版本和 PyTorch 版本
import torch
cuda_version = torch.version.cuda
# print("cuda_version :", cuda_version)
# print("torch.__version__ :", torch.__version__)

# 設置設備，0 表示第一個 GPU，-1 表示 CPU
device = 0 if torch.cuda.is_available() else -1
print("device :", device)

device : 0


In [37]:
from transformers import pipeline
model_name_or_path = "C:/Users/PARALELL/Desktop/code/DistilBert BERT/0. models/bart-large-cnn"
summarizer = pipeline("summarization", 
                      model=model_name_or_path,
                      device=device)

In [46]:
from transformers import BartForConditionalGeneration, BartTokenizer

model_name_or_path = "C:/Users/PARALELL/Desktop/code/DistilBert BERT/0. models/bart-large-cnn"
summarizer_model = BartForConditionalGeneration.from_pretrained(model_name_or_path)
summarizer_model.to(device)

# 載入 BART tokenizer
tokenizer = BartTokenizer.from_pretrained(model_name_or_path)

# def bart_large_cnnV1(text, device):
#     # Tokenize 並計算 token 數量
#     tokens = tokenizer(text, truncation=False)["input_ids"]
#     print("tokens :",len(tokens))

#     # # 設備設定（CPU或GPU）
#     # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     # summarizer_model.to(device)

#     # 分詞與編碼
#     inputs = tokenizer.encode(text, return_tensors="pt", truncation=False) # ("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
#     inputs = inputs.to(device)

#     # 模型生成摘要
#     summary_ids = summarizer_model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

#     # 解碼生成的摘要
#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

#     return summary

def bart_large_cnn(text, device):
    # 設定模型最大 token 數量
    max_tokens = 128

    # 使用 tokenizer 分段確保每段 token 數量不超過 max_tokens
    def split_text_into_chunks(text, max_tokens):
        tokens = tokenizer(text, return_tensors="pt", truncation=False)["input_ids"][0]
        chunks = []
        current_chunk = []

        for token in tokens:
            current_chunk.append(token)
            if len(current_chunk) >= max_tokens:
                chunks.append(current_chunk)
                current_chunk = []

        if current_chunk:
            chunks.append(current_chunk)

        return chunks

    # 分割文本為 token chunks
    token_chunks = split_text_into_chunks(text, max_tokens)
    #print(f"Number of token chunks: {len(token_chunks)}")

    summaries = []

    # 對每個 token chunk 生成摘要
    for chunk in token_chunks:
        inputs = torch.tensor([chunk]).to(device)

        # 模型生成摘要
        summary_ids = summarizer_model.generate(
            inputs, max_length=1024, min_length=40, num_beams=4
        ) # , length_penalty=2.0 , early_stopping=True

        # 解碼生成的摘要
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    # 將所有段落的摘要合併
    final_summary = " ".join(summaries)
    return final_summary

In [47]:
from datasets import load_dataset
imdb_train = load_dataset("imdb",split="train[:2000]+train[-2000:]") 
imdb_test = load_dataset("imdb",split="train[:500]+train[-500:]") 
imdb_val = load_dataset("imdb",split="train[500:1000]+train[-1000:-500]") 
imdb_test_val = load_dataset("imdb",split="train[2000:3000]+train[-3000:-2000]") 

In [42]:
imdb_test_val

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

In [43]:
from transformers import pipeline, DistilBertForSequenceClassification, DistilBertTokenizerFast

model_name_or_path = "C:/Users/PARALELL/Desktop/code/DistilBert BERT/0. models/DistilBert/V2/MyBestIMDBModel2"

model = DistilBertForSequenceClassification.from_pretrained(model_name_or_path,id2label = {0:"NEG",1:"POS"},label2id={"NEG":0,"POS":1})
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name_or_path)
model.to(device)
nlp_DistilBert = pipeline("sentiment-analysis",model=model,tokenizer=tokenizer,device=device)

In [39]:
def func1 (view, sentiment,device):
    # 取得原始情感分析的結果
    beforeLabel = sentiment
    print("beforeLabel :",beforeLabel)
    
    # # 進行摘要
    # result = summarizer(view, max_length=128, min_length=100, do_sample=False)

    # # 判斷摘要後的影評，其情感是否與摘要前相同    
    # summarized_text = result[0]["summary_text"]
    
    summarized_text = bart_large_cnn(view,device)
    print("-"*100)
    print("summarized_text")
    print(summarized_text)
    print("-"*100)
    summarzie_result = nlp_DistilBert(summarized_text)
    
    
    afterLabel = summarzie_result[0]["label"]
    if (afterLabel == "NEG"):
        afterLabel = 0
    else:
        afterLabel = 1
    print("afterLabel :",afterLabel)
    sentiment_score = summarzie_result[0]["score"]
    print("sentiment_score :",sentiment_score)
    
    sentimentSameOrNot = 0
    # 判斷前後情感是否相同
    if (beforeLabel == afterLabel):
        sentimentSameOrNot = 1
    else:
        sentimentSameOrNot = 0
        
    dict1 = {
        "original_sentence":view,
        "original_sentiment":beforeLabel,
        "summarized_text":summarized_text,
        "summarized_sentiment":afterLabel,
        "sentiment_score":sentiment_score,
        "sentiment_same_or_not":sentimentSameOrNot,
    }
    
    return dict1

In [48]:
# Token indices sequence length is longer than the specified maximum sequence length for this model (1223 > 512). Running this sequence through the model will result in indexing errors
dataset = imdb_test_val

i = 0
view = dataset[i]["text"]
sentiment = dataset[i]["label"]
device = 0 if torch.cuda.is_available() else -1

print(view)
print(sentiment)
print("="*100)

dict1 = func1(view, sentiment, device)


Darcy and her young daughter Pamela are heading out to the country where her mum's boyfriend Peter left his doctor's position in the city to become a writer and fix up a bed and breakfast inn. Although this inn has a terrible past and Pamela learns from one the girl's who lives in the town that a deformed witch once reside in that house. They called her the 'Tooth Fairy' as she would kill kids after getting their last baby tooth. This work on the inn, has awoken the 'Tooth Fairy'. Now she has her sights on Pamela and her last baby tooth, but if any gets in the way they face the same fate that awaits Pamela.<br /><br />This flick's old folk myth of the 'Tooth Fairy' doesn't paint her in a very generous way, as you would believe when you were a child. Don't they just love turning happy childhood memories into nightmares! Another one which did fall into the same category was "Darkness Falls (2003)". I can't compare how similar they are in the premises, because I haven't seen the latter, b

In [52]:
import pandas as pd
df1 = pd.DataFrame(columns=["original_sentence","original_sentiment","summarized_text","summarized_sentiment","sentiment_same_or_not"])
df1   

,original_sentence,original_sentiment,summarized_text,summarized_sentiment,sentiment_same_or_not


In [53]:
dataset = imdb_test_val


# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ['TORCH_USE_CUDA_DSA'] = "1"

device = 0 if torch.cuda.is_available() else -1
print("device :", device)



for i in range(0,dataset.shape[0]): # imdb_train.shape[0]
    try:    
        print("="*100)
        print(f"index = {i}")
        
        view = dataset[i]["text"]
        sentiment = dataset[i]["label"]
        
        # if (sentiment == 0): # False
        #     sentiment = "NEG"
        # if (sentiment == 1): # Ture
        #     sentiment = "POS"
        
        dict1 = func1(view, sentiment, device)
        
        new_df = pd.DataFrame([dict1])
        df1 = pd.concat([df1, new_df], ignore_index=True)
    except Exception as error_msg:
        print(f"index = {i} 出現錯誤\n", error_msg)
 

device : 0
index = 0
beforeLabel : 0
----------------------------------------------------------------------------------------------------
summarized_text
Darcy and her young daughter Pamela are heading out to the country. Her mum's boyfriend Peter left his doctor's position in the city to become a writer and fix up a bed and breakfast inn. Although this inn has a terrible past and Pamela learns from one the girl's who lives in the town that a deformed witch once reside in that house. They called her the 'Tooth Fairy' as she would kill kids after getting their last baby tooth. This flick's old folk myth of the 'Tooth Fairy' doesn't paint her in a very generous way. For a little straight to DVD film, this DTV effort looks good and has some promising images. Cory Strode and Cookie Rae Brown's story or background for this 'Tooth Fairy' character is completely bare with it leaning more towards a slasher vehicle than anything really supernatural. Low expectations are needed, as I wouldn't cl

Token indices sequence length is longer than the specified maximum sequence length for this model (971 > 512). Running this sequence through the model will result in indexing errors


----------------------------------------------------------------------------------------------------
summarized_text
Some have praised _Atlantis:_The_Lost_Empire_ as a Disney adventure for adults. I don't think so--at least not for thinking adults. The characters are weak. Even the background that each member trots out seems stock and awkward at times. Only the main character, Milo Thatch, a young Atlantis-obsessed academic voiced by Michael J. Fox, has any depth to him. Milo's search for Atlantis continues that of his grandfather who raised him. The story about how one companion, Vinny played by Don Novello, went from flower stores to demolitions totally unconvincing. While the characters were thin at best, the best part about _Atlantis_ was the voice talent. Commander Rourke loses nothing being voiced by James Garner. I'm sorry to say Leonard Nimoy's Dying King is nothing more than obligatory. The plot was the worst. The character animation has not been this bad for Disney since the 

In [54]:
df1

,original_sentence,original_sentiment,summarized_text,summarized_sentiment,sentiment_same_or_not,sentiment_score
0,Darcy and her young daughter Pamela are headin...,0,Darcy and her young daughter Pamela are headin...,0,1,0.992684
1,I was duped into seeing this movie after readi...,0,I was duped into seeing this movie after readi...,0,1,0.560388
2,Stephen J. Cannell apparently decided a few ye...,0,Stephen J. Cannell apparently decided a few ye...,0,1,0.994697
3,"When I saw this movie cover, the first thing I...",0,The film starts off with a small back story fo...,0,1,0.995472
4,"this movie is similar to Darkness Falls,and Th...",0,"this movie is similar to Darkness Falls,and Th...",0,1,0.992728
...,...,...,...,...,...,...
1962,Meryl Streep was incredible in this film. She ...,1,Meryl Streep was incredible in this film. I re...,1,1,0.992504
1963,Michael Kallio gives a strong and convincing p...,1,Michael Kallio gives a strong and convincing p...,1,1,0.992443
1964,"""Hatred of a Minute"" is a hauntingly beautiful...",1,"""Hatred of a Minute"" is a psychological thrill...",1,1,0.991857
1965,"Simply put, this is the best movie to come out...",1,The best movie to come out of Michigan since.....,1,1,0.990372


In [55]:
from datetime import datetime
now = datetime.now()
formatted_time = now.strftime("%Y.%m.%d_%H.%M.%S")  # 格式：年-月-日 時:分:秒
#print("現在時間:", formatted_time)
fileName = f"./{formatted_time}_imdb_test_val.csv"
df1.to_csv(fileName)

In [56]:
df1["sentiment_same_or_not"].value_counts()

sentiment_same_or_not
1    1724
0     243
Name: count, dtype: int64